# Real-time Implementation and Automation with Oanda 

## Recap: Historical Data, real-time Data and Orders

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
api = tpqoa.tpqoa("oanda.cfg")

In [ ]:
api.get_history(instrument = "EUR_USD", start = "2020-09-21", end = "2020-09-24",
                granularity = "H1", price = "B")

In [ ]:
api.stream_data("EUR_USD", stop = 10) 

In [ ]:
api.stop_stream()

In [ ]:
api.create_order(instrument = "EUR_USD", units = 100000)

In [ ]:
api.create_order(instrument = "EUR_USD", units = -100000)

In [ ]:
api.get_account_summary()

In [ ]:
api.get_transactions()

In [ ]:
api.print_transactions()

## Collecting and storing real-time tick data

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
api = tpqoa.tpqoa("oanda.cfg")

In [ ]:
api.stream_data("EUR_USD", stop = 10)

In [ ]:
# two steps: 1) Retrieve tick data 2) call on_sucess() method to print tick data
api.stream_data??

In [ ]:
# called by stream_data() when new data is retrieved 
api.on_success??

In [ ]:
class CloneClass(tpqoa.tpqoa):
    
    def on_success(self, time, bid, ask):
        print("Time: {} | Bid: {} | Ask:{}".format(time, bid, ask))

In [ ]:
api = CloneClass("oanda.cfg")

In [ ]:
api.get_instruments()[:5]

In [ ]:
api.stream_data("EUR_USD", stop = 5)

## Storing and resampling real-time tick data (Part 1)

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
class GetTickData(tpqoa.tpqoa):
    
    def __init__(self, config_file):
        super().__init__(config_file)
        self.tick_data = pd.DataFrame()
    
    def on_success(self, time, bid, ask):
        print(time, bid, ask)
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)

In [ ]:
td = GetTickData("oanda.cfg")

In [ ]:
td.stream_data("EUR_USD", stop = 10)

In [ ]:
td.tick_data

In [ ]:
td.tick_data.resample("1s", label = "right").last().ffill() # resample to 1s bars

In [ ]:
td = GetTickData("oanda.cfg")

In [ ]:
td.stream_data("EUR_USD", stop = 100)

In [ ]:
td.tick_data.resample("10s", label = "right").last()#.ffill()

## Storing and resampling real-time tick data (Part 2)

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
class GetTickData(tpqoa.tpqoa):
    
    def __init__(self, config_file, bar_length):
        super().__init__(config_file)
        self.bar_length = bar_length
        self.tick_data = pd.DataFrame()
    
    def on_success(self, time, bid, ask):
        print(time, bid, ask)
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]

In [ ]:
td = GetTickData("oanda.cfg", "5s")

In [ ]:
td.stream_data("EUR_USD", stop = 20)

In [ ]:
td.data

In [ ]:
td.tick_data

## Implementing a Contrarian Strategy in Real-Time (Part 1)

In [ ]:
import pandas as pd
import numpy as np
import tpqoa

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.position = 0
        self.instrument = instrument
        self.window = window
        self.bar_length = bar_length
        self.units = units
        self.tick_data = pd.DataFrame()

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s", 1, 100000)

In [ ]:
trader.window

In [ ]:
trader.stream_data(trader.instrument, stop = 10)

In [ ]:
trader.ticks

## Implementing a Contrarian Strategy in Real-Time (Part 2)

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.position = 0
        self.instrument = instrument
        self.window = window
        self.bar_length = bar_length
        self.units = units
        self.tick_data = pd.DataFrame()
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # store and resample tick data
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
        
        # prepare data & define strategy
        self.data["returns"] = np.log(self.data.mid / self.data.mid.shift(1))
        self.data["position"] = -np.sign(self.data.returns.rolling(self.window).mean())

In [ ]:
for i in range(10):
    print(i, end = " ")

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s", 1, 100000)

In [ ]:
trader.stream_data(trader.instrument, stop = 50)

In [ ]:
trader.tick_data

In [ ]:
trader.data

## Implementing a Contrarian Strategy in Real-Time (Part 3)

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.position = 0
        self.instrument = instrument
        self.window = window
        self.bar_length = bar_length
        self.units = units
        self.tick_data = pd.DataFrame()
        self.min_length = self.window + 1 # minimum number of bars required to start the trade
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # store and resample tick data
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
        
        # prepare data & define strategy
        self.data["returns"] = np.log(self.data.mid / self.data.mid.shift(1))
        self.data["position"] = -np.sign(self.data.returns.rolling(self.window).mean())
        
        # executing trades
        if len(self.data) > self.min_length - 1: # if a new bar is added: go through if/elif
            self.min_length += 1
            if self.data["position"].iloc[-1] == 1: # if signal is long
                if self.position == 0:
                    self.create_order(self.instrument, self.units) # buy 1 * units if position is neutral ("1 trade")
                elif self.position == -1:
                    self.create_order(self.instrument, self.units * 2) # buy 2 * units if position is short ("2 trades") 
                self.position = 1
            elif self.data["position"].iloc[-1] == -1: # if signal is short
                if self.position == 0:
                    self.create_order(self.instrument, -self.units) # sell 1 * units if position is neutral ("1 trade")
                elif self.position == 1:
                    self.create_order(self.instrument, -self.units * 2) # sell 2 * units if position is long ("2 trades")
                self.position = -1

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s", 1, 100000)

In [ ]:
trader.stream_data(trader.instrument, stop = 50)

In [ ]:
trader.data

In [ ]:
trader.position

In [ ]:
trader.units

In [ ]:
trader.create_order(trader.instrument, units = -trader.position * trader.units) # close Final Position

In [ ]:
trader.print_transactions(293)

## Implementing a Contrarian Strategy in Real-Time (Part 4)

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.position = 0
        self.instrument = instrument
        self.window = window
        self.bar_length = bar_length
        self.units = units
        self.tick_data = pd.DataFrame()
        self.min_length = self.window + 1
    
    def resample_data(self):
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
    
    def prepare_data(self):
        self.data["returns"] = np.log(self.data.mid / self.data.mid.shift(1))
        self.data["position"] = -np.sign(self.data.returns.rolling(self.window).mean())
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # store and resample tick data
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)
        self.resample_data()
        
        # prepare data & define strategy
        self.prepare_data()
        
        # executing trades
        if len(self.data) > self.min_length - 1:
            self.min_length += 1
            if self.data["position"].iloc[-1] == 1: # if signal is long
                if self.position == 0:
                    self.create_order(self.instrument, self.units) # buy 1 * units if position is neutral ("1 trade")
                elif self.position == -1:
                    self.create_order(self.instrument, self.units * 2) # buy 2 * units if position is short ("2 trades") 
                self.position = 1
            elif self.data["position"].iloc[-1] == -1: # if signal is short
                if self.position == 0:
                    self.create_order(self.instrument, -self.units) # sell 1 * units if position is neutral ("1 trade")
                elif self.position == 1:
                    self.create_order(self.instrument, -self.units * 2) # sell 2 * units if position is long ("2 trades")
                self.position = -1

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s", 1, 100000)

In [ ]:
order = trader.create_order(trader.instrument, 100000, suppress = True, ret = True)

In [ ]:
order

In [ ]:
order["price"]

In [ ]:
order2 = trader.create_order(trader.instrument, -100000, suppress = True, ret = True)

In [ ]:
order2 

In [ ]:
order2["pl"]

In [ ]:
order2["time"]

In [ ]:
order2["units"]

In [ ]:
order2["id"]

## Implementing a Contrarian Strategy in Real-Time (Part 5)

In [ ]:
import pandas as pd
import numpy as np
import tpqoa

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.position = 0
        self.instrument = instrument
        self.window = window
        self.bar_length = bar_length
        self.units = units
        self.tick_data = pd.DataFrame()
        self.min_length = self.window + 1
        self.profits = [] # store p&l for all trades
    
    def resample_data(self):
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
    
    def prepare_data(self):
        self.data["returns"] = np.log(self.data.mid / self.data.mid.shift(1))
        self.data["position"] = -np.sign(self.data.returns.rolling(self.window).mean())
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # store and resample tick data
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)
        self.resample_data()
        
        # prepare data & define strategy
        self.prepare_data()
        
        # executing trades
        if len(self.data) > self.min_length - 1:
            self.min_length += 1
            if self.data["position"].iloc[-1] == 1:
                if self.position == 0:
                    order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                    self.report_trade(order, "GOING LONG")
                elif self.position == -1:
                    order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                    self.report_trade(order, "GOING LONG")
                self.position = 1
            elif self.data["position"].iloc[-1] == -1: 
                if self.position == 0:
                    order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                    self.report_trade(order, "GOING SHORT")
                elif self.position == 1:
                    order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                    self.report_trade(order, "GOING SHORT")
                self.position = -1
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s", 1, 100000)

## Implementing a Contrarian Strategy in Real-Time (Part 6)

In [ ]:
trader.stream_data(trader.instrument, stop = 50) # trading (50 ticks)
close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                  suppress = True, ret = True) # close Final Position
trader.report_trade(close_order, "GOING NEUTRAL") # report Final Trade

In [ ]:
trader.data

In [ ]:
trader.tick_data

In [ ]:
trader.profits

## Code Efficiency

In [ ]:
import pandas as pd
import tpqoa

In [ ]:
class GetTickData(tpqoa.tpqoa):
    
    def __init__(self, config_file, bar_length):
        super().__init__(config_file)
        self.bar_length = bar_length
        self.tick_data = pd.DataFrame()
    
    def on_success(self, time, bid, ask):
        if self.ticks % 5 == 0:
            df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                              index = [pd.to_datetime(time)])
            self.tick_data = self.tick_data.append(df)
            self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]

In [ ]:
td = GetTickData("oanda.cfg", "1min")

In [ ]:
td.stream_data("EUR_USD", stop = 50)

In [ ]:
td.tick_data

In [ ]:
len(td.tick_data)

In [ ]:
td.data

## Working with historical data and real-time tick data (Part 1)

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta

In [ ]:
instrument = "EUR_USD"

In [ ]:
now = datetime.utcnow()
now

In [ ]:
api = tpqoa.tpqoa("oanda.cfg")

In [ ]:
api.get_history(instrument = "EUR_USD", start = "2020-12-07", end = "2020-12-08",
                granularity = "M1", price = "B")

In [ ]:
now = now - timedelta(microseconds = now.microsecond)
now

In [ ]:
yesterday = now - timedelta(days = 1)
yesterday

In [ ]:
df = api.get_history(instrument = "EUR_USD", start = yesterday, end = now,
                     granularity = "M1", price = "M")

In [ ]:
df

In [ ]:
df.index = df.index.tz_localize("UTC")

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.position = 0
        self.instrument = instrument
        self.window = window
        self.bar_length = bar_length
        self.units = units
        self.tick_data = pd.DataFrame()
        self.hist_data = None
        self.min_length = None
        self.raw_data = None
        self.data = None
        self.profits = []
    
    def get_most_recent(self, days = 5):
        now = datetime.utcnow()
        now = now - timedelta(microseconds = now.microsecond)
        past = now - timedelta(days = days)
        df = self.get_history(instrument = self.instrument, start = past, end = now,
                               granularity = "M1", price = "M").c.dropna().to_frame()
        df.rename(columns = {"c":self.instrument}, inplace = True)
        df.index = df.index.tz_localize("UTC")
        df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
        self.hist_data = df.copy()
        self.min_length = len(self.hist_data) + 1
        
        #...

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "2min", 20, 100000) # 1. create Trader Object

In [ ]:
trader.get_most_recent(days = 5) # 2. Get Historical Data

In [ ]:
trader.hist_data

In [ ]:
# 3. Stream Live Data, Merge with Historical Data, Trading ...

## Working with historical data and real-time tick data (Part 2)

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.position = 0
        self.instrument = instrument
        self.window = window
        self.bar_length = bar_length
        self.units = units
        self.tick_data = pd.DataFrame()
        self.hist_data = None
        self.min_length = None
        self.raw_data = None
        self.data = None
        self.profits = []
    
    def get_most_recent(self, days = 5):
        now = datetime.utcnow()
        now = now - timedelta(microseconds = now.microsecond)
        past = now - timedelta(days = days)
        df = self.get_history(instrument = self.instrument, start = past, end = now,
                               granularity = "M1", price = "M").c.dropna().to_frame()
        df.rename(columns = {"c":self.instrument}, inplace = True)
        df.index = df.index.tz_localize("UTC")
        df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
        self.hist_data = df.copy()
        self.min_length = len(self.hist_data) + 1
    
    def resample_and_join(self):
        self.raw_data = self.hist_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1]) 
    
    def prepare_data(self):
        df = self.raw_data.copy()
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        self.data = df.copy()
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # store and resample tick data and join with historical data
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)
        self.resample_and_join()
        
        # only if full new bar has been added:
        if len(self.raw_data) > self.min_length - 1:
            self.min_length += 1
            self.prepare_data()
            
            # order and trades...
            # if self.data["position"].iloc[-1] == 1:
                #if self.position == 0:
            
            
            
            

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "2min", 20, 100000) # 1. create Trader Object

In [ ]:
trader.get_most_recent(days = 5) # 2. Get Historical Data

In [ ]:
trader.stream_data("EUR_USD", stop = 200) # 3. Stream Live Data, Merge with Historical, Trading

In [ ]:
trader.hist_data

In [ ]:
trader.tick_data

In [ ]:
trader.raw_data

In [ ]:
trader.data

In [ ]:
trader.min_length

####################################################################################################

(you can find the corresponding Video for the following lines of code in __Section 23__: Cloud Deployment (AWS) | Scheduling Trading Sessions | Full Automation)

## How to Time/Schedule the End of a Trading Session (Simple Contrarian)

In [ ]:
import pandas as pd
import numpy as np
import tpqoa
import datetime as dt

In [ ]:
dt.datetime.utcnow().time() # current utc time

In [ ]:
dt.time(17, 45) # desired end of trading session (in utc time)

In [ ]:
if dt.datetime.utcnow().time() >= dt.time(12, 45):
    print("Stop the Trading Session!")

In [ ]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.position = 0
        self.instrument = instrument
        self.window = window
        self.bar_length = bar_length
        self.units = units
        self.tick_data = pd.DataFrame()
        self.min_length = self.window + 1
        self.profits = [] # store p&l for all trades
    
    def resample_data(self):
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
    
    def prepare_data(self):
        self.data["returns"] = np.log(self.data.mid / self.data.mid.shift(1))
        self.data["position"] = -np.sign(self.data.returns.rolling(self.window).mean())
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # store and resample tick data
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)
        self.resample_data()
        
        # prepare data & define strategy
        self.prepare_data()
        
        # executing trades
        if len(self.data) > self.min_length - 1:
            self.min_length += 1
            if self.data["position"].iloc[-1] == 1:
                if self.position == 0:
                    order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                    self.report_trade(order, "GOING LONG")
                elif self.position == -1:
                    order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                    self.report_trade(order, "GOING LONG")
                self.position = 1
            elif self.data["position"].iloc[-1] == -1: 
                if self.position == 0:
                    order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                    self.report_trade(order, "GOING SHORT")
                elif self.position == 1:
                    order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                    self.report_trade(order, "GOING SHORT")
                self.position = -1
        
        # stop
        if dt.datetime.utcnow().time() >= dt.time(17, 45):
            self.stop_stream = True    
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")

In [ ]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s", 1, 100000)

In [ ]:
trader.stream_data(trader.instrument)
close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                  suppress = True, ret = True) # close Final Position
trader.report_trade(close_order, "GOING NEUTRAL") # report Final Trade